In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd
#from pandasql import sqldf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import dateutil.parser
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
df=pd.read_csv('../input/internet-articles-data-with-users-engagement/articles_data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

**top performers**

In [ ]:
fig=px.pie(df,values='top_article',names='source_name',title='news article value',hole=0.5)
fig.update_traces(textposition='inside',textinfo='percent+label')
fig.show()

In [ ]:
fig=px.pie(df,values='engagement_share_count',names='source_name',title='news popularity',hole=0.5)
fig.update_traces(textposition='inside',textinfo='percent+label')
fig.show()

In [ ]:
top=df['source_name'].value_counts().head()
top

In [ ]:
top_list=top.index.tolist()
df_sample=df[df["source_name"].isin(top_list)]

In [ ]:
def topList(df,n,main_field,grouping_fields,sumi=False):
  allColumns=grouping_fields+[main_field]
  titles=df.dropna(subset=allColumns)
  titles=titles[allColumns]
  if sumi:
    titles=titles.groupby(grouping_fields).sum()
  titles=titles.sort_values([main_field],ascending=False)
  titles.rename(columns={main_field:main_field+'sum'},inplace=True)
  return titles.head(n)


In [ ]:
topList(df_sample,10,main_field='engagement_reaction_count',grouping_fields=['title','source_name'])

In [ ]:
topList(df_sample,10,main_field='top_article',grouping_fields=['author'],sumi=True)

# **Engagement Per Publisher**

In [ ]:
def engagement_publisher(df):
  arr=[]
  publishers=list(df['source_name'].unique())
  engagement=['engagement_reaction_count','engagement_share_count','engagement_comment_count']
  for i in engagement:
    df_eng=df.groupby(['source_name'])[i].agg('sum')
    arr.append((i,df_eng.tolist()))
  return publishers,arr
publishers,engagement_publisher=engagement_publisher(df_sample)

fig=go.Figure(go.Bar(x=publishers,y=engagement_publisher[0][1],name=engagement_publisher[0][0]))
for engagement_name,engagement_results in engagement_publisher[1:]:
  fig.add_trace(go.Bar(x=publishers,y=engagement_results,name=engagement_name))
fig.update_layout(barmode='relative',xaxis={'categoryorder':'array'})
fig.show()

In [ ]:
df_temp = df_sample \
    .groupby(['source_id'])['source_name','top_article', 'engagement_reaction_count', 'engagement_comment_count','engagement_share_count', 'engagement_comment_plugin_count']\
    .agg('sum')

In [ ]:
df_temp=(df_temp-df_temp.min())/(df_temp.max()-df_temp.min())
pl1=sns.heatmap(df_temp,cmap="YlGnBu",robust=True)
pl1.tick_params(labelsize=14)
pl1.figure.set_size_inches((5,5))

# Sources with Maximum Reaction

In [ ]:
fig=px.pie(df,values='engagement_reaction_count',names='source_name',title='reactions',hole=0.5)
fig.update_traces(textposition='inside',textinfo='percent+label')
fig.show()